<a href="https://colab.research.google.com/github/yasudakn/umaibar/blob/master/Latent_Me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Save a Copy (File -> Save a copy in Drive) to run

# Finding yourself in the latent space of StyleGAN

Dmitry Nikitko (puzer) wrote [the original code](https://github.com/Puzer/stylegan) for this notebook which extends the work released by NVidia on StyleGAN. I have modified and annotated it to make it easier to use in Colab. After running through this notebook you should have a StyleGAN generated image (or set of images) which closely match photos of you that you've uploaded.

You'll also have a `npy` file which is the latent code (or location) of your image in the StyleGAN latent space.

In [1]:
!git clone https://github.com/Puzer/stylegan

Cloning into 'stylegan'...
remote: Enumerating objects: 105, done.
remote: Total 105 (delta 0), reused 0 (delta 0), pack-reused 105
Receiving objects: 100% (105/105), 10.24 MiB | 11.73 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [2]:
%cd stylegan
# Use the version this notebook was built with
!git checkout c3fb250c65840c8837ded78e34485227755c2473

/content/stylegan
Note: checking out 'c3fb250c65840c8837ded78e34485227755c2473'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at c3fb250 Update README.md


In [0]:
!mkdir raw_images aligned_images generated_images latent_representations

## Add your image(s)

Upload your image using the Sidebar now. 

To open the sidebar select "View" and then "Table of Contents".

The sidebar should now be open. Click the "Files" tab.

Before uploading make sure:

1.   The image(s) you're using can be opened by PIL (jpg, png, etc)
2.   The images are larger than 1024x1024. Preferably significantly larger so the aligner can crop out a high resolution section of the image containing your face.
3.   Your face in the image is well lit and facing the camera (for best results)

Click ''Upload" in the sidebar and select the images you want to upload from your computer.

Note: All files uploaded in this manner end up in the root of the file tree. We'll move them into the correct spot next.

In [0]:
# e.g. mv ../me.jpg raw_images/
!mv ../*.jpg raw_images/

## Align your images

In [0]:
!python align_images.py raw_images aligned_images

This should produce an image in `aligned_images/` for every image in `raw_images/`.

It's a good idea to check that this process worked by using the Files browser to download each aligned image and make sure it looks reasonable. If you encounter scrambled images it might be because your original raw images are too small.

## Search for your latent self

The script `encode_images.py` will minimize the perceptual loss between generated images from StyleGAN and each of the images you've uploaded. (By default this happens one at a time)

I've had good results at 1000 iterations and it's best to check the general quality before coming back and ramping up the number of iterations to produce a high-quality latent.

Higher quality comes at a cost of course. 10000 iterations will take about **one hour** for one image.

**NOTE:** You may get a warning about the GPU memory limit when running this script. Don't worry it will still complete.

In [0]:
!python encode_images.py aligned_images/ generated_images/ latent_representations/ --iterations 1000

## Download Your Results

After the above cell has finished writing there should be an image in `generated_images/` for each image in `aligned_images/`.

You can right-click and download each of these images to see your final latent self.

### Latent Representation

You can also download the `npy` files in the `latent_representations/` directory. Each of those is a serialized numpy array which contains the (18, 512) array encoding the point in latent space which corresponds to the generated image. Which you can open with `latent = np.load('filename.npy')`

### Change your Smile, Gender, or Age

Once your latent representation has been generated and saved you can explore the volume around it through latent vectors. Puzer has provided vectors for Smile, Gender and Age so you can see what you look like as your latent self varies along those axes.

Run the following cells.

In [0]:
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
URL_FFHQ = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ'

tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)

generator = Generator(Gs_network, batch_size=1, randomize_noise=False)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    return img #img.resize((256, 256))

def move_and_show(latent_vector, direction, coeffs):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
        ax[i].imshow(generate_image(new_latent_vector))
        ax[i].set_title('Coeff: %0.1f' % coeff)
    [x.axis('off') for x in ax]
    plt.show()

In [0]:
# Loading already learned representations
me = np.load('latent_representations/my_01.npy')

In [0]:
# Loading already learned latent directions
smile_direction = np.load('ffhq_dataset/latent_directions/smile.npy')
gender_direction = np.load('ffhq_dataset/latent_directions/gender.npy')
age_direction = np.load('ffhq_dataset/latent_directions/age.npy')

# In general it's possible to find directions of almost any face attributes: position, hair style or color ... 
# Additional scripts for doing so will be realised soon

# Smile transformation

In [0]:
move_and_show(me, smile_direction, [-0.5, 0, 0.5])

# Gender transformation

In [0]:
move_and_show(me, gender_direction, [-1, 0, 1])

# Age transformation

In [0]:
move_and_show(me, age_direction, [-0.5, 0, 0.5])

In [0]:
  Gs = Gs_network
  
  # Pick latent vector.
  rnd = np.random.RandomState(10)  # seed = 10
  latents0 = rnd.randn(1, Gs.input_shape[1])
  latents1 = rnd.randn(1, Gs.input_shape[1])
  latents2 = rnd.randn(1, Gs.input_shape[1])
  latents3 = rnd.randn(1, Gs.input_shape[1])
  latents4 = rnd.randn(1, Gs.input_shape[1])
  latents5 = rnd.randn(1, Gs.input_shape[1])
  latents6 = rnd.randn(1, Gs.input_shape[1])

  num_split = 39  # 2つのベクトルを39分割
  for i in range(40):
      latents = latents6+(latents0-latents6)*i/num_split
      # Generate image.
      fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
      images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

      # Save image.
      os.makedirs(config.result_dir, exist_ok=True)
      png_filename = os.path.join(config.result_dir, 'photo'+'{0:04d}'.format(i)+'.png')
      PIL.Image.fromarray(images[0], 'RGB').save(png_filename)

In [0]:
from PIL import Image
import glob
 
files = sorted(glob.glob('results/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('stylegan.gif', save_all=True, 
               append_images=images[1:], 
               duration=200, loop=0)

In [0]:
url_ffhq        = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)
 
_Gs_cache = dict()
 
def load_Gs(url):
    if url not in _Gs_cache:
        with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
            _G, _D, Gs = pickle.load(f)
        _Gs_cache[url] = Gs
    return _Gs_cache[url]
 
# ----------------  Style mixing -------------------
 
def draw_style_mixing_figure(png, Gs, w, h, src_seeds, dst_seeds, style_ranges):
    print(png)
    src_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in src_seeds)
    
    d1 = np.random.RandomState(503).randn(Gs.input_shape[1])  # seed = 503 のベクトルを取得
    d2 = np.random.RandomState(888).randn(Gs.input_shape[1])  # seed = 888 のベクトルを取得
    
    dx = (d2 - d1)/3  #　３分割で補間
    steps = np.linspace(0,3,4)  # stepsに[0,1,2,3] を代入
    dst_latents = np.stack((d1+dx*step) for step in steps)  # dst_latents にベクトルを４つスタック
        
    src_dlatents = Gs.components.mapping.run(src_latents, None) # [seed, layer, component]
    dst_dlatents = Gs.components.mapping.run(dst_latents, None) # [seed, layer, component]
    src_images = Gs.components.synthesis.run(src_dlatents, randomize_noise=False, **synthesis_kwargs)
    dst_images = Gs.components.synthesis.run(dst_dlatents, randomize_noise=False, **synthesis_kwargs)
    
    canvas = PIL.Image.new('RGB', (w * (len(src_seeds) + 1), h * 5), 'white')
    for col, src_image in enumerate(list(src_images)):
        canvas.paste(PIL.Image.fromarray(src_image, 'RGB'), ((col + 1) * w, 0))
    for row, dst_image in enumerate(list(dst_images)):
        canvas.paste(PIL.Image.fromarray(dst_image, 'RGB'), (0, (row + 1) * h))  
        row_dlatents = np.stack([dst_dlatents[row]] * len(src_seeds))
        row_dlatents[:, style_ranges[row]] = src_dlatents[:, style_ranges[row]]
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)
        for col, image in enumerate(list(row_images)):
            canvas.paste(PIL.Image.fromarray(image, 'RGB'), ((col + 1) * w, (row + 1) * h)) 
             
    png_filename = os.path.join(config.result_dir, 'style_mix.png')
    canvas.save(png_filename)
 
# --------------- main -----------------

In [9]:

tflib.init_tf()
os.makedirs(config.result_dir, exist_ok=True)
draw_style_mixing_figure(os.path.join(config.result_dir, 'style_mix.png'), load_Gs(url_ffhq), w=1024, h=1024, 
                                               src_seeds=[11,100,701,583], dst_seeds=[888,829,1898,1733,1614,845], 
                                               style_ranges=[range(1, 10)]*4)  # style_mixingのレンジ指定


results/style_mix.png
